<a href="https://colab.research.google.com/github/Gilf641/Foreign_ExchangeRate_Forecast/blob/master/S12_AssignmentSolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive', force_remount=True)



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/My Drive/EVA4/updLib2/evaLibrary')
!ls '/content/drive/My Drive/EVA4/updLib2/evaLibrary'


AlbTestTransforms.py   customNet.py    execute.py    resNet.py
AlbTrainTransforms.py  cyclicLR.py     Gradcam.py    rohan_library.py
albumentations.py      DataLoaders.py  LR_Finder.py
all.py		       displayData.py  __pycache__


In [3]:
import execute
from resNet import ResNet18
import displayData as display
import Gradcam as gdc
import albumentations as alb
# import DataLoaders as loader
import AlbTestTransforms
import AlbTrainTransforms
import LR_Finder as lrf
import cyclicLR as clr


  pip install torch-lr-finder -v --global-option="amp"


In [12]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

--2020-06-24 16:17:49--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  17.5MB/s    in 17s     

2020-06-24 16:18:06 (13.9 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [16]:
!unzip -q tiny-imagenet-200.zip && ls tiny-imagenet-200

replace tiny-imagenet-200/words.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
test  tiny-imagenet-200  train	val  wnids.txt	words.txt


In [17]:
defaultPath = "/content/tiny-imagenet-200/"
trainPath = defaultPath+"train"
testPath = defaultPath+"test"
valPath = defaultPath+"val"


**DataLoader part**

In [6]:
import torchvision.datasets as datasets
import torch.utils.data as data
import os

In [7]:
num_workers = {"train":4, "val":0, "test":0}
transforms = {"train": AlbTrainTransforms.train_transforms(), "val": AlbTestTransforms.test_transforms(), "test": AlbTestTransforms.test_transforms()}

image_datasets = {x: datasets.ImageFolder(os.path.join(defaultPath, x), transforms[x]) for x in ["train", "val", "test"]}
dataLoaders = {x: data.DataLoader(image_datasets[x], batch_size=128, shuffle=True, num_workers=num_workers[x], pin_memory=True) for x in ["train", "val", "test"]}
sizeofDataset = {x: len(image_datasets[x]) for x in ["train", "val", "test"]}




FileNotFoundError: ignored

In [14]:
import matplotlib.pyplot as plt

how_many_to_plot = 100

train_loader = dataLoaders["train"]

def show_images_horizontally(images, labels=[], un_normalize=False, fig_size=(15, 7)):
    """Show images in jupyter notebook horizontally w/ labels as title.
    Parameters
    ----------
    images: pytorch Tensor of shape (#images, #channels, height, width)
    labels: pytorch Tensor of shape (#images, label)
        labels should be encoded in number like 0, 1 .. and so on.
    un_normalize: bool
        indicate whether to perform unnormalization operation for rendering.
    fig_size: tuple
    """

    fig = plt.figure(figsize=fig_size)
    num_imgs = images.shape[0]
    for i in range(num_imgs):
        fig.add_subplot(1, num_imgs, i + 1)

        # render image tensor
        img = images[i]
        npimg = img.numpy()
        if un_normalize:
            npimg = npimg / 2 + 0.5
        npimg = np.transpose(npimg, (1, 2, 0))

        # generate label as title
        if labels:
            plt.title(lookup_label[labels[i][0]])
        plt.imshow(npimg, cmap='Greys_r')
        plt.axis('off')

# for i, batch in enumerate(train_loader, start=1):
#   image, label = batch
#   print(label)
# plt.figure(figsize = (75 ,75))
# for i, batch in enumerate(train_loader, start=1):
#   image, label = batch
#   print(label)
#   # print(max(label[:]))
#   # show_images_horizontally(image, un_normalize=True)

 

NameError: ignored

In [9]:
from torchsummary import summary
import torch
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
print('Device set to ', device)

model = ResNet18().to(device)
summary(model, input_size=(3,32,32))

Device set to  cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-1

In [ ]:
import torch.optim as optim
CUDA_LAUNCH_BLOCKING=1 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-7, weight_decay=1e-2)

# lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
# lr_finder.range_test(train_loader, end_lr=100, num_iter=100, step_mode="exp")



In [ ]:
clr.lr_rangetest(device, model, train_loader, criterion, 0.0001, 0.001, 40)

In [116]:
import os
import csv
import random
import requests
import zipfile
import numpy as np
from io import BytesIO
from PIL import Image
from torch.utils.data import Dataset


class tinyImageDataset:

    def __init__(self, 
                path,
                train = True, 
                download=True, 
                splitRatio=0.70, 
                random_seed=110, 
                transform=None):
        self.path=path
        self.train=train
        self.download=download
        self.splitRatio=splitRatio
        self.random_seed=random_seed
        self.transform=transform

        # download the dataset
        if download:
            self.downloadDataset()
        
        if splitRatio > 1:
            raise ValueError("train_split must be less than 1")

        self._classID = self._mapID_to_className()  
        self.data, self.target = self._loadData(self.train)
        print("data ", self.data)
        # print("target ", len(self.target))

        # shuffle the dataset
        self._imageIndex = np.arange(len(self.target))
        np.random.seed(random_seed)
        np.random.shuffle(self._imageIndex)

        # Split the data using Image indices
        print("ImageIndices", len(self._imageIndex))
        splitValue = int(len(self._imageIndex)*self.splitRatio) 
        print("SplitValue", splitValue)
        self._imageIndex = self._imageIndex[:splitValue] if self.train else self._imageIndex[splitValue:]



    def __len__(self):
        return len(self._imageIndex) 


    def __getitem__(self, index):
        image_index = self._imageIndex[index]
        image = self.data[image_index]   

        if self.transform:
            image = self.transform(image)

        return image, self.target[image_index]

    
    def __repr__(self):
        head = 'TinyImageNet Dataset'
        body = ['Number of datapoints: {}'.format(self.__len__())]
        if self.path is not None:
            body.append('Root Location: {}'.format(self.path))
        body += [f'Split: {"Train" if self.train else "Test"}']
        if hasattr(self, 'transforms') and self.transforms is not None:
            body += [repr(self.transforms)]
        lines = [head] + [' ' * 4 + line for line in body]
        return '\n'.join(lines)


    def tinyClasses(self):
        return tuple(cls[1]['name']  for cls in sorted(self._classID, key=lambda x:x[1]['id']))     

    def _mapID_to_className(self):
        with open(os.path.join(self.path, 'tiny-imagenet-200/wnids.txt')) as f:
            class_ids = {x[:-1]: '' for x in f.readlines()} 

        with open(os.path.join(self.path, 'tiny-imagenet-200/words.txt')) as f:
            class_id = 0
            for line in csv.reader(f, delimiter='\t'):
                if line[0] in class_ids:
                    class_ids[line[0]] = {"name": line[1], "id": class_id}
                    class_id += 1
        
        return class_ids 

    def _loadImage(self, image_path):
        img = Image.open(image_path)

        if img.mode == 'L':
            img_array = np.array(img)#convert image to np array
            print(img_array.shape) 
            img = np.stack((img_array,)*3, axis=-1)
            img = Image.fromarray(img.astype('uint8'), 'RGB') #converting the np image back normal

        return img
    
    def _loadData(self, train = True):
        data = []
        target = []
        
        if train:
          train_path = os.path.join(self.path, "tiny-imagenet-200/train")
          for classDir in os.listdir(train_path):
              trainData_path = os.path.join(train_path, classDir+"/images")
              for image in os.listdir(trainData_path):
                  data.append(self._loadImage(os.path.join(trainData_path, image)))
                  target.append(self._classID[classDir]["id"])
                    
      
        else:
          val_path = os.path.join(self.path, 'tiny-imagenet-200/val')
          valImages_path = os.path.join(val_path, 'images')
          with open(os.path.join(val_path, 'val_annotations.txt')) as f:
              for line in csv.reader(f, delimiter='\t'):
                  data.append(self._loadImage(os.path.join(valImages_path, line[0])))
                  target.append(self._classID[line[1]]['id'])
        
        print('datalen: ', len(data))
        print('targetlen: ', len(target))
        return data, target
    
    def downloadDataset(self):
        if not os.path.exists(self.path):
            r = requests.get('http://cs231n.stanford.edu/tiny-imagenet-200.zip', stream=True)
            zip_ref = zipfile.ZipFile(BytesIO(r.content))
            zip_ref.extractall(os.path.dirname(self.path))
            zip_ref.close()

            # # Move file to appropriate location
            # os.rename(os.path.join(os.path.dirname(self.path), 'tiny-imagenet-200'), self.path)
        else:
            print('Files already downloaded.')

def loader(trainData, valData, train=True):
        loader_args = {
            'batch_size': 128,
            'num_workers': 4,
            'pin_memory': True,
            'cuda': True
        }

        return data_loader(
            trainData, **loader_args
        ) if train else data_loader(valData, **loader_args)


def data_loader(data, shuffle=True, batch_size=1, num_workers=1, pin_memory=True, cuda=True):
    
    loader_args = {
        'shuffle': shuffle,
        'batch_size': batch_size
    }

    # If GPU exists
    if cuda:
        loader_args['num_workers'] = num_workers
        loader_args['pin_memory'] = True
    
    return torch.utils.data.DataLoader(data, **loader_args)


In [117]:
from torchvision import datasets, transforms
# albTrain = AlbTrainTransforms.train_transforms()
# albTest = AlbTestTransforms.test_transforms()
mainTransform = transforms.Compose([transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [118]:
defaultPath = "/content/tiny-imagenet-200/"
train_dataset = tinyImageDataset(defaultPath, 
                train=True,
                download=True,                    
                splitRatio=0.70, 
                random_seed=110, 
                transform=mainTransform)


val_dataset = tinyImageDataset(defaultPath,
                train=False,
                download=True, 
                random_seed=110, 
                transform=mainTransform)


Files already downloaded.
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(6

RuntimeError: ignored

#Not sure about the split

In [110]:
trainLoader = loader(train_dataset, val_dataset, train=True)
valLoader = loader(train_dataset, val_dataset, train=False)


In [111]:
import torch.optim as optim
import torch.nn as nn
CUDA_LAUNCH_BLOCKING=1 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-7, weight_decay=1e-2)

lr_finder =lrf.LRFinder(model, optimizer, criterion, device="cuda")
lr_finder.range_test(trainLoader, end_lr=100, num_iter=100, step_mode="exp")



RuntimeError: ignored